# sofa_calculation

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
pd.set_option('display.max_rows', 300)

code_token = pd.read_parquet('code_token.parquet')

In [3]:
# MAP value replacement
offset_group = code_token.groupby(['stay_id', 'offset'])
replacement_rows = []

for (stay_id, offset), group in offset_group :
    if 220052 not in group['itemid'].values : 
        if (220050 in group['itemid'].values) and (220051 in group['itemid'].values) : 
            replacement_map_value = (group.loc[group['itemid']==220050,'value'].iloc[0] + (2 * group.loc[group['itemid']==220051, 'value'].iloc[0])) / 3
            replacement_rows.append({'stay_id': stay_id, 'offset': offset, 'itemid': 220052, 'value': replacement_map_value})
            print('MAP value has been replaced | stay_id : ', stay_id, '| offset : ', offset)
        else : 
            pass
    else : 
        pass

code_token = pd.concat([code_token, pd.DataFrame(replacement_rows)], ignore_index=True)

# sofa score indicator filtering
required_ids = [220224, 223835, 227457, 225690, 220739, 223900, 223901, 220615]
optional_ids = [220052, 221662, 221653, 221289, 221906]
grouped = code_token.groupby('stay_id')['itemid'].apply(list).reset_index()

def filter_rows(row):
    required_present = all(id_ in row['itemid'] for id_ in required_ids)
    optional_present = any(id_ in row['itemid'] for id_ in optional_ids)
    return required_present and optional_present

filtered_stay_ids = grouped[grouped.apply(filter_rows, axis=1)]['stay_id']
filtered_timeline = code_token[code_token['stay_id'].isin(filtered_stay_ids)].reset_index(drop=True)

# sofa 평가 요소 데이터프레임 생성
sofa_item_id = [220224, 223835, 227457, 225690, 220739, 223900, 223901, 220615, 220052, 221662, 221653, 221289, 221906]
col = ['stay_id', 'offset', 'itemid', 'value']
stay_ids = filtered_timeline['stay_id'].unique()

sofa_data = []

for stay_id in stay_ids:
    stay_offsets = filtered_timeline[filtered_timeline['stay_id'] == stay_id]['offset'].unique()
    for offset in stay_offsets:
        for item_id in sofa_item_id:   
            sofa_data.append([stay_id, offset, item_id, None])

sofa_df = pd.DataFrame(sofa_data, columns=col).sort_values(['stay_id','offset']).reset_index(drop=True)
merged_sofa_df = pd.merge(sofa_df, filtered_timeline, on=['stay_id', 'offset', 'itemid'], how='left', suffixes = ('','_test'))
merged_sofa_df['value'] = merged_sofa_df['value_test']
merged_sofa_df.drop(columns=['value_test'], inplace=True)

# forward fill(inputevents X, chartevents O)
chartevents_ids = [220224, 223835, 227457, 225690, 220739, 223900, 223901, 220615, 220052]

for chartevents_id in chartevents_ids:
    merged_sofa_df.loc[merged_sofa_df['itemid'] == chartevents_id, 'value'] = merged_sofa_df.groupby(['stay_id', 'itemid'])['value'].ffill()

MAP value has been replaced | stay_id :  30757476 | offset :  4340
MAP value has been replaced | stay_id :  30757476 | offset :  5954
MAP value has been replaced | stay_id :  31316840 | offset :  22564
MAP value has been replaced | stay_id :  32136798 | offset :  32820
MAP value has been replaced | stay_id :  32554129 | offset :  8
MAP value has been replaced | stay_id :  34499716 | offset :  1951
MAP value has been replaced | stay_id :  34617352 | offset :  100
MAP value has been replaced | stay_id :  35514836 | offset :  182


In [4]:
offset_group = merged_sofa_df.groupby(['stay_id', 'offset'])
labels_data = []

for (stay_id, offset), group in tqdm(offset_group, desc = "sofa score 계산 작업 처리 중", ncols = 80, ascii = ' =', leave=True):
    row = {'stay_id': stay_id, 'offset': offset}
    # sofa element의 required_ids 중에서 하나라도 결측치가 있는 경우
    if group.loc[group['itemid'].isin(required_ids), 'value'].isnull().any():
        pass
    # required_ids는 모두 있고, optional_ids 중 최소 한 개 이상이 있는 경우
    elif group.loc[group['itemid'].isin(required_ids), 'value'].notnull().all() and group.loc[group['itemid'].isin(optional_ids), 'value'].notnull().any():

        # respiratory(PaO2/FiO2) (호흡 기능 - 동맥 산소 분압 대 흡입 산소 분율 비율)
        respirationa_value = group.loc[group['itemid'] == 220224, 'value'].iloc[0] / (group.loc[group['itemid'] == 223835, 'value'].iloc[0] * 0.01)
        
        if respirationa_value >= 400:
            respirationa_label = 0
        elif 300 <= respirationa_value < 400:
            respirationa_label = 1
        elif 200 <= respirationa_value < 300:
            respirationa_label = 2
        elif 100 <= respirationa_value < 200:
            respirationa_label = 3
        else:
            respirationa_label = 4
        row['respirationa_label'] = respirationa_label
        
        # coagulation (응고계 - 혈소판 농도)
        platelets = group.loc[group['itemid']==227457, 'value'].iloc[0]
        
        if platelets >= 150 : 
            platelets_label = 0
        elif 100 <= platelets < 150 :
            platelets_label = 1
        elif 50 <= platelets < 100 : 
            platelets_label = 2
        elif 20 <= platelets < 50 : 
            platelets_label = 3
        else : 
            platelets_label = 4
        row['platelets_label'] = platelets_label
        
        # liver(Bilirubin) (간 기능 - 빌리루빈)
        bilirubin = group.loc[group['itemid']==225690, 'value'].iloc[0]

        if bilirubin < 1.2 : 
            bilirubin_label = 0
        elif 1.2 <= bilirubin < 2.0 : 
            bilirubin_label = 1
        elif 2.0 <= bilirubin < 6.0 : 
            bilirubin_label = 2
        elif 6.0 <= bilirubin < 12.0 : 
            bilirubin_label = 3
        else : 
            bilirubin_label = 4
        row['bilirubin_label'] = bilirubin_label
        
        # cardiovascular (심혈관계 - 저혈압 예방을 위해 필요한 혈관 활성제 양)
        mean_arterial_pressure = group.loc[group['itemid']==220052, 'value'].iloc[0]
        dopamine = group.loc[group['itemid']==221662, 'value'].iloc[0]
        dobutamine = group.loc[group['itemid']==221653, 'value'].iloc[0]
        epinephrine = group.loc[group['itemid']==221289, 'value'].iloc[0]
        norepinephrine = group.loc[group['itemid']==221906, 'value'].iloc[0]

        if (dopamine > 15) or (epinephrine > 0.1) or (norepinephrine > 0.1):
            cardiovascular_label = 4
        elif (dopamine > 5) or (epinephrine <= 0.1) or (norepinephrine <= 0.1):
            cardiovascular_label = 3
        elif (dopamine <= 5) or (pd.notna(dobutamine)):
            cardiovascular_label = 2
        elif mean_arterial_pressure < 70:
            cardiovascular_label = 1
        elif mean_arterial_pressure >= 70:
            cardiovascular_label = 0
        row['cardiovascular_label'] = cardiovascular_label

        # Glasgow Coma Score (신경계 - 글래스고 혼수 점수)
        cns = group.loc[group['itemid']==220739, 'value'].iloc[0] + group.loc[group['itemid']==223900, 'value'].iloc[0] + group.loc[group['itemid']==223901, 'value'].iloc[0]

        if cns == 15 : 
            cns_label = 0
        elif 13 <= cns < 15 : 
            cns_label = 1
        elif 10 <= cns < 13 : 
            cns_label = 2
        elif 6 <= cns < 10 :
            cns_label = 3
        else :
            cns_label = 4
        row['cns_label'] = cns_label
        
        # renal(Creatinine) (신장 기능 - 혈중 크레아티닌)
        creatinine = group.loc[group['itemid']==220615, 'value'].iloc[0]
        if 0 <= creatinine < 1.2 : 
            creatinine_label = 0
        elif 1.2 <= creatinine < 2.0 : 
            creatinine_label = 1
        elif 2.0 <= creatinine < 3.5 : 
            creatinine_label = 2
        elif 3.5 <= creatinine < 5.0 : 
            creatinine_label = 3
        else : 
            creatinine_label = 4
        row['creatinine_label'] = creatinine_label
        
        sofa_score = respirationa_label + platelets_label + bilirubin_label + cardiovascular_label + cns_label + creatinine_label
        row['sofa_score'] = sofa_score

        labels_data.append(row)

labels_df = pd.DataFrame(labels_data)
new_sofa_df = pd.merge(merged_sofa_df, labels_df, on = ['stay_id', 'offset'], how='left')

def sofa_range(value):
    if value >= 0 and value <= 5 :
        return 0
    elif value >= 6 and value <= 11 :
        return 1
    elif value >= 12 and value <= 18 :
        return 2
    elif value >= 19 and value <= 24 : 
        return 3
    else : 
        return -1

new_sofa_df['sofa_category'] = new_sofa_df['sofa_score'].apply(sofa_range)
new_sofa_df['sofa_score'] = new_sofa_df['sofa_score'].fillna(-1)

# # 원본 데이터와 조인
# fixed_code_timeline = pd.merge(code_token, new_sofa_df, on=['stay_id', 'offset', 'itemid', 'value'], how='left')
# fixed_code_timeline.drop(columns=['respirationa_label', 'platelets_label', 'bilirubin_label', 'cardiovascular_label', 'cns_label', 'creatinine_label'], inplace=True)
# fixed_code_timeline[fixed_code_timeline['stay_id']==30213599].tail(27)

sofa score 계산 작업 처리 중: 100%|=========| 960/960 [00:00<00:00, 1197.12it/s]


## label_df 확인

In [5]:
new_sofa_df[new_sofa_df['sofa_category']!=-1]['stay_id'].value_counts()

stay_id
32359580    897
38197705    793
32136798    650
35526828    520
37293400    507
34100191    377
37919901    338
38740124    299
30757476    260
34575919    247
31361200    234
33630048    221
32128372    208
31959184    208
38383343    182
33072499    169
30213599    169
31316840    156
33060379    130
38875437    130
39625056    130
35214014    130
38554095    130
35009126    117
37093652    104
34617352    104
36558922     91
37153661     65
33035972     65
35479615     26
39635619     26
Name: count, dtype: int64

In [6]:
print('필터링 이후 잔여 stay_id 수 : ', new_sofa_df['stay_id'].nunique())

필터링 이후 잔여 stay_id 수 :  32


In [7]:
new_sofa_df[new_sofa_df['stay_id']==31316840].iloc[364:377,:]

,stay_id,offset,itemid,value,respirationa_label,platelets_label,bilirubin_label,cardiovascular_label,cns_label,creatinine_label,sofa_score,sofa_category
949,31316840,15124,220224,94.7500,2.0,0.0,0.0,3.0,4.0,2.0,11.0,1
950,31316840,15124,223835,40.5560,2.0,0.0,0.0,3.0,4.0,2.0,11.0,1
951,31316840,15124,227457,233.0000,2.0,0.0,0.0,3.0,4.0,2.0,11.0,1
952,31316840,15124,225690,0.3000,2.0,0.0,0.0,3.0,4.0,2.0,11.0,1
953,31316840,15124,220739,1.5000,2.0,0.0,0.0,3.0,4.0,2.0,11.0,1
954,31316840,15124,223900,1.0000,2.0,0.0,0.0,3.0,4.0,2.0,11.0,1
955,31316840,15124,223901,1.7500,2.0,0.0,0.0,3.0,4.0,2.0,11.0,1
956,31316840,15124,220615,3.4800,2.0,0.0,0.0,3.0,4.0,2.0,11.0,1
957,31316840,15124,220052,72.3650,2.0,0.0,0.0,3.0,4.0,2.0,11.0,1
958,31316840,15124,221662,NaN,2.0,0.0,0.0,3.0,4.0,2.0,11.0,1


In [8]:
extraction = new_sofa_df[new_sofa_df['stay_id']==31316840]
extraction.to_excel('stay_id_31316840.xlsx')